# Plots by Sebastian
## Plotting genres part of movies pr. year

### Importing libraries

In [1]:
import matplotlib.pyplot as plt 
from data_gen import gen_df
import numpy as np
import ipywidgets as widgets
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

### Generating dataframe from data-file

In [46]:
filename = 'imdb.csv'

df = gen_df(filename)
df['year'] = df['year'].astype(int)
df.head()

,index,tid,title,imdbRating,ratingCount,duration,year,nrOfWins,nrOfNominations,nrOfPhotos,...,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western,decade
0,2200,tt0011565,The Penalty (1920),7.6,1095.0,1.500000,1920,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1920s
1,502,tt0011841,MÌ_dchenlos (1920),8.1,3134.0,2.416667,1920,0.0,0.0,18.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1920s
2,12832,tt0011865,Irrwege einer Ehe (1920),8.2,1042.0,1.500000,1920,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1920s
3,12148,tt0010323,Das Cabinet des Dr. Caligari (1920),8.1,29379.0,1.300000,1920,0.0,0.0,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1920s
4,12368,tt0011071,Buster Keaton als StrÌ_fling (1920),7.1,1474.0,0.333333,1920,0.0,0.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1920s


### Plotting genres as part of total movies at the year
These are generated as:
$$
\text{Genre share} = \frac{\text{Total movies in genre in year}}{\text{Total movies in year}}
$$

In [49]:
# Generate genre list
genre_list = list(df)[13:41]

In [50]:
def _plot_1(df,genre):
    
    df['count'] = 1
    df['movies_year'] = df.groupby('year')['count'].transform(lambda x: x.sum())  

    # Plot year sum of different genres
    for i in genre:
    #i = genre

        df[f'{i}_year'] = df.groupby('year')[i].transform(lambda x: x.sum())
        df[f'{i}_share'] = df[f'{i}_year']/df['movies_year']

        y_share = df.groupby('year')[f'{i}_share'].first()

        #ax = plt.subplot(2,2,j)
        y_share.plot(kind='line', sharex='col', sharey='row')

        plt.xlabel('year')
        plt.ylabel('share of movies')
        plt.title(i)
        
    #plt.show()
    
def plot_1(df):
    
    widgets.interact(_plot_1,
                    df = widgets.fixed(df),
                    genre = widgets.SelectMultiple(
                        options = genre_list,
                        description = 'Genres',
                        disabled = False))
    
plot_1(df)


interactive(children=(SelectMultiple(description='Genres', options=('Action', 'Adult', 'Adventure', 'Animation…

## Listing top/bottom 5 movies per decade and finding their original titles from IMDb
To do this we sort the datasets on imdbRating and select top 10 an bottom 10.
From this we pull out the id's, and use these to find original titles from IMDb. This is donw by using the request-module, which acceses the IMDb-website and finds the title.

In [194]:
# Condition on minimum number of ratings
I = df['ratingCount'] >= 5000

# Sort top and bottom movies by decade
top = df.sort_values(['decade', 'imdbRating'], ascending=False)[I];
top5_decade = top.groupby('decade').head(5)
bottom = df.sort_values(['decade', 'imdbRating'], ascending=True)[I];
bottom5_decade = bottom.groupby('decade').head(5)

# Get IMDb id's
top5_id = top5_decade.tid.tolist()
bottom5_id = bottom5_decade.tid.tolist()
id5_list = top5_id + bottom5_id 

# Initiate list to get names from IMDb
eng_names = []

# Get original names from IMDb using requests
for i,id in enumerate(id5_list):
    test = requests.get('https://www.imdb.com/title/' + id)
    soup = bs(test.text,'html.parser')
    if soup.find('div',class_='originalTitle') != None:
        eng_names.append(soup.find('div',class_='originalTitle').text)
    else:
        eng_names.append(soup.find('h1').text)

c:\python\python37-32\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
c:\python\python37-32\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [195]:
df_merge_1 = pd.DataFrame(i for i in eng_names)
df_merge_2 = pd.DataFrame(i for i in id5_list)
df_merge = df_merge_1.merge(df_merge_2, left_index=True, right_index=True)
df_merge = df_merge.rename(columns={'0_x': 'eng_title', '0_y': 'tid'})

text = ''

top_bottom = df.merge(df_merge, how='inner', on='tid')
top_bottom['eng_title'] = top_bottom['eng_title'].str.split('\(').str[0]
#grouped = top_bottom[['eng_title','imdbRating','decade']].sort_values('imdbRating', ascending=False).reset_index(drop=True).groupby('decade')


for name, group in grouped:
    text += '\n\n'
    text += f'The top 5 movies of the {name} are: '
    name_list = group.eng_title.tolist()
    rating_list = group.imdbRating.tolist()
    for j,i in enumerate(name_list):
        if j < 5:
            text += '\n'
            text += f'{i:<40} with rating: {rating_list[j]}'
        if j == 5:
            text += '\n\n'
            text += f'The bottom 5 movies of the {name} are: \n'
            text += f'{i:<40} with rating: {rating_list[j]}'
        if j > 5:
            text += '\n'
            text += f'{i:<40} with rating: {rating_list[j]}'   
            
print(text)
#top_bottom = top_bottom[['title','eng_title','imdbRating','decade','year']].reset_index(drop=True)
#top_bottom
#top_bottom.drop(columns='index', inplace=True)



The top 5 movies of the 1920s are: 
Metropolis                               with rating: 8.4
The Kid                                  with rating: 8.4
Chelovek s kino-apparatom                with rating: 8.4
Sunrise: A Song of Two Humans            with rating: 8.4
Safety Last!                             with rating: 8.3

The bottom 5 movies of the 1920s are: 
Das Cabinet des Dr. Caligari             with rating: 8.1
Nosferatu, eine Symphonie des Grauens    with rating: 8.0
Bronenosets Potemkin                     with rating: 8.0
Un chien andalou                         with rating: 7.9
The Phantom of the Opera                 with rating: 7.8

The top 5 movies of the 1930s are: 
City Lights                              with rating: 8.7
Modern Times                             with rating: 8.6
M - Eine Stadt sucht einen Mörder        with rating: 8.5
Mr. Smith Goes to Washington             with rating: 8.4
It Happened One Night                    with rating: 8.3

The bottom 5 m